Import Libraries

In [ ]:
import czifile
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
from skimage import exposure, filters, measure
from scipy.ndimage import binary_fill_holes
import scipy.ndimage as ndi
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
import pandas as pd
import os

Define Functions

In [ ]:
def mask_near_border(mask, distance=5):
    """
    Checks if a binary mask comes within a specified distance of the image borders.
    """
    if np.any(mask[:distance, :]) or np.any(mask[-distance:, :]) or \
       np.any(mask[:, :distance]) or np.any(mask[:, -distance:]):
        return True
    return False

def visualize_mask(image_squeezed, mask, basename, region_index):
    """
    Visualizes the mask applied to the channels and saves the result.
    """
    channel1_masked = np.where(mask, 0, image_squeezed[1])
    channel2_masked = np.where(mask, 0, image_squeezed[2])

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(channel1_masked, cmap='gray')
    plt.title('Channel 1 Masked')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(channel2_masked, cmap='gray')
    plt.title('Channel 2 Masked')
    plt.axis('off')

    plt.savefig(f'{basename}/visualize_mask_region{region_index}.png')

def create_mfi_ratio_map(image_squeezed, dilated_mask, region, basename, i):
    """
    Create a pixel-by-pixel rendering of the MFI ratio map for a specific region in an image.

    Parameters:
    - image_squeezed: np.ndarray, 3D image array with the shape (channels, height, width).
    - dilated_mask: np.ndarray, 2D binary mask array indicating the region of interest.
    - region: RegionProperties, region object containing bounding box information.
    - basename: str, base name for saving the output image.
    - i: int, region index for saving the output image with unique names.

    Returns:
    - mfi_ratio_map: np.ndarray, the MFI ratio map for the specified region.
    """

    # Create the pixel-by-pixel rendering
    dilated_mask_region = dilated_mask[region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]] 

    # Apply the dilated mask to each channel within the region
    channel1_masked = image_squeezed[1, region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]] * dilated_mask_region 
    channel2_masked = image_squeezed[2, region.bbox[0]:region.bbox[2], region.bbox[1]:region.bbox[3]] * dilated_mask_region

    # Calculate the MFI ratio map
    with np.errstate(divide='ignore', invalid='ignore'):
        pixel_ratios = np.where(channel1_masked != 0, channel2_masked / channel1_masked, 0) 

    mfi_ratio_map = np.zeros(region.image.shape, dtype=float)
    mfi_ratio_map[dilated_mask_region] = pixel_ratios[dilated_mask_region] 

    # Normalize and visualize the MFI ratio map
    cmap = 'viridis'
    positive_mfi_ratio_map = mfi_ratio_map.copy()
    positive_mfi_ratio_map[positive_mfi_ratio_map <= 0] = np.nanmin(positive_mfi_ratio_map[positive_mfi_ratio_map > 0])  # Replace non-positive values with the smallest positive value
    norm = LogNorm(vmin=np.nanmin(positive_mfi_ratio_map), vmax=np.nanmax(positive_mfi_ratio_map))  

    plt.figure(figsize=(6, 6))
    plt.imshow(positive_mfi_ratio_map, cmap=cmap, norm=norm)
    plt.colorbar(label='MFI Ratio')
    plt.axis('off')

    # Save the plot
    output_path = fr'{basename}\mfi_ratio_map_region{i}.png'
    plt.savefig(output_path)
    plt.close()

    return mfi_ratio_map

def segment_cells(image, basename):
    """
    Segments cells from the image and calculates mean fluorescence intensity (MFI) ratios.
    """
    image_squeezed = np.squeeze(image)
    first_frame = image_squeezed[0]
    green_channel = gaussian(first_frame, sigma=2)
    optimal_threshold = threshold_otsu(first_frame)

    cells_thresholded = green_channel > optimal_threshold
    cells_edges = binary_erosion(binary_dilation(sobel(cells_thresholded), disk(2)), disk(1))
    labeled_image = label(cells_edges)
    min_size_threshold = 1000

    intensity_ratios, cfps, yfps = [], [], []
    for i, region in enumerate(regionprops(labeled_image), start=1):
        if region.area < min_size_threshold:
            continue
        mask = labeled_image == region.label
        if mask_near_border(mask):
            continue

        dilated_mask = binary_fill_holes(mask)
        visualize_mask(image_squeezed, ~dilated_mask, basename, i)
        create_mfi_ratio_map(image_squeezed, dilated_mask, region, basename, i)

        channel1_masked = image_squeezed[1] * dilated_mask
        channel2_masked = image_squeezed[2] * dilated_mask

        mean_intensity_channel1 = np.mean(channel1_masked[dilated_mask])
        mean_intensity_channel2 = np.mean(channel2_masked[dilated_mask])

        if mean_intensity_channel2 != 0:
            intensity_ratios.append(mean_intensity_channel2 / mean_intensity_channel1)
        cfps.append(mean_intensity_channel1)
        yfps.append(mean_intensity_channel2)

    return labeled_image, intensity_ratios, cfps, yfps


def analyze_czi_file(file_path, basename):
    """
    Analyzes a CZI file from a FRET experiment.
    """
    image = czifile.imread(file_path)
    labeled_image, intensity_ratios, cfps, yfps = segment_cells(image, basename)
    return pd.DataFrame({
        "Filename": [basename] * len(intensity_ratios),
        "Cell": range(len(intensity_ratios)),
        "CFP_MFI": cfps,
        "YFP_FRET_MFI": yfps
    })

Main Script 

In [ ]:
folder_path = r"test_images"
all_data = []

for well_image in os.listdir(folder_path):
    if well_image.lower().endswith(".czi"):
        well_image_path = os.path.join(folder_path, well_image)
        well_image_base_name = os.path.splitext(well_image)[0]
        cwd = os.getcwd() #get current working directory
        well_image_dir = os.path.join(cwd, well_image_base_name)
        os.makedirs(well_image_dir, exist_ok=True)
        df = analyze_czi_file(well_image_path, well_image_base_name)
        all_data.append(df)

# Combine and save the DataFrame
combined_df = pd.concat(all_data, ignore_index=True)
combined_df.to_excel(r'combined_FRET_data.xlsx', index=False)